In [95]:
import pandas as pd
from PIL import Image
import cv2
import numpy as np
import math

In [96]:
original_images = context.catalog.load("express_expense")

[06/04/23 10:41:16] INFO     Loading data from 'express_expense' (PartitionedDataSet)...        ]8;id=362833;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=537264;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [97]:
points_auto = context.catalog.load("homography_points_segmentation")

                    INFO     Loading data from 'homography_points_segmentation' (CSVDataSet)... ]8;id=429235;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=576913;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [98]:
points_auto

,index,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y
0,1000,181,198,662,180,709,999,111,999
1,1001,296,491,318,487,326,503,298,504
2,1002,83,25,415,181,370,999,2,999
3,1003,183,68,597,47,585,990,156,942
4,1004,83,71,602,77,662,906,17,889
...,...,...,...,...,...,...,...,...,...
176,1193,36,0,302,0,319,347,24,347
177,1194,23,45,195,57,250,423,13,448
178,1195,122,0,579,0,623,999,65,999
179,1197,25,117,286,101,309,449,21,436


In [99]:
points_auto = points_auto.rename(columns={'index': 'Image'})
#points_auto["Image"] = points_auto['Image'].str.replace("-receipt.jpg", "").astype('uint32')
#points_auto["Image"] = points_auto['Image'].str.replace("batched_", "").astype('uint32')
#points_auto["Image"] = points_auto['Image'].str.replace("batched_", "").str.replace("mask.*", "", regex=True).astype('uint32')
points_auto = points_auto.set_index("Image")

In [100]:
points_auto

,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y
Image,,,,,,,,
1000,181,198,662,180,709,999,111,999
1001,296,491,318,487,326,503,298,504
1002,83,25,415,181,370,999,2,999
1003,183,68,597,47,585,990,156,942
1004,83,71,602,77,662,906,17,889
...,...,...,...,...,...,...,...,...
1193,36,0,302,0,319,347,24,347
1194,23,45,195,57,250,423,13,448
1195,122,0,579,0,623,999,65,999


In [101]:
points_manual = pd.read_csv("../data/01_raw/homography.csv", sep=";")

In [102]:
points_manual['P1X'] = points_manual['P1'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [103]:
points_manual['P1Y'] = points_manual['P1'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [104]:
points_manual['P2X'] = points_manual['P2'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [105]:
points_manual['P2Y'] = points_manual['P2'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [106]:
# o P3 está invertido com o P4 na detecção manual, não sei por qual motivo

In [107]:
points_manual['P3X'] = points_manual['P4'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [108]:
points_manual['P3Y'] = points_manual['P4'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [109]:
points_manual['P4X'] = points_manual['P3'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [110]:
points_manual['P4Y'] = points_manual['P3'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [111]:
points_manual = points_manual.drop(columns=['P1', 'P2', 'P3', 'P4'])

In [112]:
points_manual = points_manual.set_index("Image")

In [113]:
missing = set(points_manual.index) - set(points_auto.index)
missing

{1006,
 1015,
 1016,
 1024,
 1038,
 1044,
 1062,
 1066,
 1076,
 1087,
 1097,
 1139,
 1146,
 1147,
 1187,
 1188,
 1196,
 1199}

In [114]:
points_manual = points_manual.loc[points_auto.index]

In [115]:
points_manual

,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y
Image,,,,,,,,
1000,182,199,661,193,699,929,119,914
1001,83,77,545,90,576,788,72,824
1002,76,110,413,179,355,998,1,981
1003,177,92,596,92,594,932,159,943
1004,83,94,603,101,659,851,21,875
...,...,...,...,...,...,...,...,...
1193,34,1,303,1,320,345,22,344
1194,22,44,194,53,251,419,13,445
1195,117,7,583,7,623,996,65,994


In [116]:
dp1 = np.sqrt(pow(points_auto['P1X'] - points_manual['P1X'], 2) + pow(points_auto['P1Y'] - points_manual['P1Y'], 2))
dp2 = np.sqrt(pow(points_auto['P2X'] - points_manual['P2X'], 2) + pow(points_auto['P2Y'] - points_manual['P2Y'], 2))
dp3 = np.sqrt(pow(points_auto['P3X'] - points_manual['P3X'], 2) + pow(points_auto['P3Y'] - points_manual['P3Y'], 2))
dp4 = np.sqrt(pow(points_auto['P4X'] - points_manual['P4X'], 2) + pow(points_auto['P4Y'] - points_manual['P4Y'], 2))

In [117]:
dt = np.array(np.ravel(np.concatenate((dp1, dp2, dp3, dp4)).reshape(-1)))

In [118]:
dt.max()

614.8601467000443

In [119]:
dt.min()

0.0

In [120]:
dt.mean()

42.20879157351953

In [121]:
np.median(dt)

7.035533905932738

In [122]:
dt.std()

110.55695620736348

In [123]:
np.argmax(dt)

124